In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
def miss_values(df):
    for column in df:
        # Test whether column has null value
        if len(df[column].apply(pd.isnull).value_counts()) > 1:
            print(column+" has missing value")
            #if column is numeric, fill null with mean
            if df[column].dtype in ('int64','float64'):
                df[column] = df[column].fillna(df[column].mean())
            else:
                df[column] = df[column].fillna("unknown")

In [ ]:
def factor_encoding(df):
    for column in df:
        if df[column].dtype == 'object':
            df = df.merge(pd.get_dummies(data=df[column],prefix=column),right_index=True,left_index=True)
            del df[column]
    return df

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
train['source'] = 1.0
test['source'] = 0.0
alls = pd.concat([train,test],ignore_index=True)
miss_values(alls)
alls = factor_encoding(alls)

In [ ]:
train = alls[alls['source'] == 1.0]
train.drop(['source'],inplace=True,axis=1)
test = alls[alls['source'] == 0.0]
test.drop(['source'],inplace=True,axis=1)

In [ ]:
x = train.drop(['SalePrice'],axis=1)
Y = train['SalePrice']
x_test = test.drop(['SalePrice'],axis=1)
x_train, x_vali, Y_train, Y_vali = train_test_split(x, Y, test_size=0.25, random_state=42)

In [ ]:
para_grid = {
      'loss' : ['ls','lad','huber'],
      'max_depth':[i for i in range(1,10)],
      'max_features' : ['auto','sqrt','log2']
}

In [ ]:
gbr = GradientBoostingRegressor()
clf = GridSearchCV(estimator=gbr, param_grid=para_grid,scoring='neg_mean_squared_error')
clf.fit(x, Y)

In [ ]:
clf.best_params_ 

In [ ]:
gridSearch = pd.DataFrame.from_dict(clf.cv_results_,orient='columns')

In [ ]:
gridSearch[0:10]

In [ ]:
auto_only = gridSearch[gridSearch['param_max_features']=='auto']
line_ls_train, = plt.plot(auto_only[auto_only['param_loss']=='ls']['param_max_depth'],auto_only[auto_only['param_loss']=='ls']['mean_train_score'],'g--',label='ls')
line_lad_train, = plt.plot(auto_only[auto_only['param_loss']=='lad']['param_max_depth'],auto_only[auto_only['param_loss']=='lad']['mean_train_score'],'r--',label='lad')
line_huber_train, = plt.plot(auto_only[auto_only['param_loss']=='huber']['param_max_depth'],auto_only[auto_only['param_loss']=='huber']['mean_train_score'],'b--',label='huber')
line_ls_test, = plt.plot(auto_only[auto_only['param_loss']=='ls']['param_max_depth'],auto_only[auto_only['param_loss']=='ls']['mean_test_score'],'g',label='ls')
line_lad_test, = plt.plot(auto_only[auto_only['param_loss']=='lad']['param_max_depth'],auto_only[auto_only['param_loss']=='lad']['mean_test_score'],'r',label='lad')
line_huber_test, = plt.plot(auto_only[auto_only['param_loss']=='huber']['param_max_depth'],auto_only[auto_only['param_loss']=='huber']['mean_test_score'],'b',label='huber')
plt.legend(handles=[line_ls_train,line_ls_test,line_lad_train,line_lad_test,line_huber_train,line_huber_test])
plt.ylabel('Square-Root MSE')
#plt.title('Tuning the hiper paremeter Alpha')
plt.show()

In [ ]:
ls_only = gridSearch[gridSearch['param_loss']=='ls']
line_auto_train, = plt.plot(ls_only[ls_only['param_max_features']=='auto']['param_max_depth'],ls_only[ls_only['param_max_features']=='auto']['mean_train_score'],'g--',label='auto')
line_sqrt_train, = plt.plot(ls_only[ls_only['param_max_features']=='sqrt']['param_max_depth'],ls_only[ls_only['param_max_features']=='sqrt']['mean_train_score'],'r--',label='sqrt')
line_log2_train, = plt.plot(ls_only[ls_only['param_max_features']=='log2']['param_max_depth'],ls_only[ls_only['param_max_features']=='log2']['mean_train_score'],'b--',label='log2')
line_auto_test, = plt.plot(ls_only[ls_only['param_max_features']=='auto']['param_max_depth'],ls_only[ls_only['param_max_features']=='auto']['mean_test_score'],'g',label='auto')
line_sqrt_test, = plt.plot(ls_only[ls_only['param_max_features']=='sqrt']['param_max_depth'],ls_only[ls_only['param_max_features']=='sqrt']['mean_test_score'],'r',label='sqrt')
line_log2_test, = plt.plot(ls_only[ls_only['param_max_features']=='log2']['param_max_depth'],ls_only[ls_only['param_max_features']=='log2']['mean_test_score'],'b',label='log2')
plt.legend(handles=[line_auto_train,line_auto_test,line_sqrt_train,line_sqrt_test,line_log2_train,line_log2_test])
plt.ylabel('Square-Root MSE')
#plt.title('Tuning the hiper paremeter Alpha')
plt.show()

In [ ]:
gridSearch

In [ ]:
clf.best_params_ 

In [ ]:
#clf = linear_model.Lasso(alpha=20,max_iter=10000,normalize=True)
clf = GradientBoostingRegressor(max_depth=2)
clf.fit(x,Y)
test_predicted = clf.predict(x_test)

In [ ]:
test['SalePrice'] = test_predicted
result = test[['Id','SalePrice']]

In [ ]:
result.to_csv('submission.csv',index=False)